# Implementing a deep convolutional GAN

In [1]:
#!pip install tensorflow-datasets

In [2]:
#TO LOCATION OF AN INSTALLED PACKAGE

#!pip show tensorflow-datasets

In [3]:
# HOW TO CHECK PACKAGE ENVIRONMENT

#import sys
#print(sys.executable)
#import tensorflow_datasets as tfds
#print(tfds.__version__)

In [57]:
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.layers import *
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tqdm import tqdm

DEFINE AN ALIAS FOR THE AUTOTUNE SETTING, WHICH WE'LL USE LATER
TO DETERMINE THE NUMBER OF PARALLEL CALLS WHEN PROCESSING THE IMAGES
IN THE DATASET:

In [58]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

DEFINE A DCGAN() CLASS TO ENCAPSULATE OUR IMPLEMENTATION.
THE CONSTRUCTOR CREATE THE DISCRIMINATOR, GENERATOR, LOSS FUNCTION,
AND THE RSPECTIVE OPTIMIZERS FOR BOTH SUB-NETWORKS:

In [102]:
class DCGAN(object):
    def __init__(self):
        self.loss = BinaryCrossentropy(from_logits = True)
        self.generator = self.create_generator()
        self.discriminator = self.create_discriminator()
        self.generator_opt = Adam(learning_rate = 1e-4)
        self.discriminator_opt = Adam(learning_rate = 1e-4)

        '''
        Defne a static method to create the generator network. It reconstructs a 28x28x1
        image from an input tensor of 100 elements. Notice the use of transposed
        convolutions (Conv2DTranspose) to expand the output volumes as we go deeper
        into the network. Also, notice the activation is 'tanh', which means the outputs
        will be in the range [-1, 1]:
        '''
    @staticmethod
    def create_generator(negative_slope = 0.3):
        input = Input(shape = (100,))
        x = Dense(units = 7 * 7 *256,
                 use_bias = False) (input)
        x = LeakyReLU(negative_slope = negative_slope) (x)
        x = BatchNormalization() (x)

        x = Reshape((7, 7, 256)) (x)

        # Add the frst transposed convolution block, with 128 flters:
        x = Conv2DTranspose(filters = 128,
                           strides = (1, 1),
                           kernel_size = (5, 5),
                           padding = "same",
                           use_bias = False) (x)
        x = LeakyReLU(negative_slope = negative_slope) (x)
        x = BatchNormalization() (x)

        # Create the second transposed convolution block, with 64 flters:
        x = Conv2DTranspose(filters = 64,
                           strides = (2, 2),
                           kernel_size = (5, 5),
                           padding = "same",
                           use_bias = False) (x)
        x = LeakyReLU(negative_slope = negative_slope) (x)
        x = BatchNormalization() (x)

        # add the last transposed convolution block, with only one filter, corresponding to
        #the output of the network:
        x = Conv2DTranspose(filters = 1,
                           strides = (2, 2),
                           kernel_size = (5, 5),
                           padding = "same",
                           use_bias = False) (x)
        output = Activation("tanh") (x)

        return Model(input, output)
# Defne a static method to create the discriminator. 
# This architecture is a regular CNN:
    @staticmethod
    def create_discriminator(negative_slope = 0.3, dropout = 0.3):
        input = Input(shape = (28, 28, 1))
        x = Conv2D(filters = 64, 
                  kernel_size = (5, 5),
                  strides = (2, 2),
                  padding = "same") (input)
        x = LeakyReLU(negative_slope = negative_slope) (x)
        x = Dropout(rate = dropout) (x)

        x = Conv2D(filters = 128,
                  kernel_size = (5, 5),
                  strides = (2, 2),
                  padding ="same") (x)
        x = LeakyReLU(negative_slope = negative_slope) (x)
        x = Dropout(rate = dropout) (x)

        x = Flatten() (x)
        output = Dense(units = 1) (x)

        return Model(input, output)
    # Defne a method to calculate the discriminator's loss, 
    # which is the sum of the real and fake losses:
    def discriminator_loss(self, real, fake):
        real_loss = self.loss(tf.ones_like(real), real)
        fake_loss = self.loss(tf.zeros_like(fake), fake)

        return real_loss + fake_loss

    # Defne a method to calculate the generator's loss:
    def generator_loss(self, fake):
        return self.loss(tf.ones_like(fake), fake)

        # Defne a method to perform a single training step. 
        # We'll start by generating a vector of random Gaussian noise:
    @tf.function
    def train_step(self, images, batch_size):
        noise = tf.random.normal((batch_size, noise_dimension))

        # pass the random noise to the generator to produce fake images:
        with tf.GradientTape() as gen_tape, tf.GradientTape() as dis_tape:
            generated_images = self.generator(noise,
                                             training = True)

            # Pass the real and fake images to the discriminator 
            # and compute the losses of both sub-networks:
            real = self.discriminator(images, 
                                     training = True)
            fake = self.discriminator(generated_images,
                                     training = True)

            gen_loss = self.generator_loss(fake)
            disc_loss = self.discriminator_loss(real,
                                               fake)
        # Compute the gradients:
        generator_grad = gen_tape.gradient(gen_loss,
                 self.generator.trainable_variables)
        discriminator_grad = dis_tape.gradient(disc_loss,
                                                   self.discriminator.trainable_variables)

        # Apply the gradients using the respective optimizers:
        opt_args = zip(generator_grad,
                      self.generator.trainable_variables)
        self.generator_opt.apply_gradients(opt_args)

        opt_args = zip(discriminator_grad,
                      self.discriminator.trainable_variables)
        self.discriminator_opt.apply_gradients(opt_args)

    # defne a method to train the whole architecture. Every 10 epochs,
    # we will plot the images the generator produces in order to 
    # visually assess their quality:
    def train(self, dataset, test_seed, epochs, batch_size):
                     for epoch in tqdm(range(epochs)):
                        for image_batch in dataset:
                            self.train_step(image_batch,
                                           batch_size)
                        if epoch == 0 or epoch % 10 == 0:
                            generate_and_save_images(self.generator,
                                                    epoch,
                                                    test_seed)
                                                    

                

        

Defne a function to produce new images, and then save a 4x4 mosaic of them to
disk:

In [103]:
def generate_and_save_images(model, epoch, test_input):
    predictions = model(test_input, training = False)

    plt.figure(figsize = (4, 4))
    for i in range(predictions.shape[0]):
        plt.subplot(4, 4, i + 1)
        image = predictions[i, :, :, 0] * 127.5 + 127.5
        image = tf.cast(image, tf.uint8)
        plt.imshow(image, cmap = "gray")
        plt.axis("off")

    plt.savefig(f"{epoch}.png")
    plt.show()

DEFINE A FUNCTION TO SCALE THE IMAGES THAT COME FROM THE EMNIST
DATASET TO THE [-1, 1] INTERVAL

In [104]:
def process_image(input):
    image = tf.cast(input["image"], tf.float32)
    image = (image - 127.5) / 127.5
    return image

LOAD THE EMNIST DATASET USING TFDS. WE'LL USE ONLY THE "TRAIN" SPLIT
WHICH CONTAINS MORE THAN 600,000 IMAGES.
WE'LL ALSO MAKE SURE TO SCALE EACH IMAGE USING THE "TANH" RANGE

In [105]:
BUFFER_SIZE = 1000
BATCH_SIZE = 512
train_dataset = (tfds.load("emnist", split = "train")
                .map(process_image,
                    num_parallel_calls = AUTOTUNE)
                .shuffle(BUFFER_SIZE)
                .batch(BATCH_SIZE))

CREATE A TEST SEED THAT WILL BE USED THROUGHOUT THE TRAINING
OF THE DCGAN TO GENERATE IMAGES:

In [106]:
noise_dimension = 100
num_examples_to_generate = 16
seed_shape = (num_examples_to_generate,
             noise_dimension)
test_seed = tf.random.normal(seed_shape)

INSTANTIATE AND TRAIN A DCGAN() INSTANCE FOR 100 EPOCHS

In [108]:
EPOCHS = 100
dcgan = DCGAN()
dcgan.train(train_dataset, test_seed, EPOCHS, BATCH_SIZE)